## Parameters in BioCRNpyler
BioCRNpyler has a flexible parameter system allowing for models to be rapidly produced using very few parameters common across many reactions and then refined so that each mechanisms and components have unique reaction parameters derived from the literature or experiments. This parameter system is designed to work automatically, so all users have to do is specify a parameter file of the proper format or a parameter dictionary which can be loaded into Mixtures and Components using the parameter_file and/or parameters (for dictionaries) keywords. Parameters from multiple sources can be used together.

Parameters passed into a Component supercede mixture level parameters in reactions produced by that Component.

#### Parameter File Overview:
Parameter files can be .tsv (or .txt) or .csv. The first line of the file should contain column headings. The following headings are required (in any order): mechanism_id, part_id, param_name, param_val (spaces can be substituted for underscores and headings are not case sensitive). mechanism_id is the name of the Mechanism or the kind of mechanism that will use this parameter, for example "transcription" or  "transcription_mm" for Mechalis-Menten transcription would go in this column. part_id refers to the name supplied by the Component that will use this mechanism, for example "ptet" for a tet repressed promoter and "ptet_leak" for leak reactions of that promoter. param_name refers to the name of the model parameter, for example "ktx", "kb", or "ku". The value of these columns is case sensitive and underscores are different from spaces. All these parameter headings (if they are not empty) are required to be alpha-numeric (plus single internal underscores) and start with a letter.


#### Viewing Parameters Used in a Model
The easiest way to examine which parameter values are used in a Model and where BioCRNpyler found them is with the prett_print function of a ChemicalReactionNetwork or a Reaction with show_rates and show_keys both set to True.
    
    CRN.prett_print(show_rates = True, show_keys = True)

show_rates toggles whether reaction rates are shown. The parameter values used in the rates will be shown below the rate function. show_keys toggles whether the ParameterKey used to search and find these rates is displayed. search_key is the key the Mechanism and component searched for in order to find the rate. found_key is the key the parameter was found in after defaulting.



#### Parameter Dictionary Overview:
A parameter dictionary should have the following forms (note that part_id and mechanism can be None):

Using ParameterKeys (named tuples):

    parameters = {ParameterKey(mechanism = "mechanism_name", part_id = "part_id", name = "parameter_name"):param_val ...} #using ParameterKeys
    
Using tuples or even just strings (for default parameters) shorthand is also valid:

    parameters = {("mechanism_name", "part_id", "parameter_name"):val1, "k":default_val ...} 

#### Parameter Value Defaulting:
Parameters can be passed into Components and Mixtures as both files and/or dictionaries. Parameters into Components will be used by reactions created by that Component:

    Component(... parameters = parameters, parameter_file = "file_name" ...)

If a Component cannot find its own parameters, it will use parameters passed into the Mixture:

    Mixture(... parameters = parameters, parameter_file = "file_name" ...)

At both the Mixture and Component levels, BioCRNpyler does nto require an exact keyword match. When Mechanisms are called by Components to produce reactions, they will initially look for

    ParameterKey(mechanism = "mechanism_name", part_id = "part_id", name = "parameter_name") --> param_val. 

If that particular parameter key cannot be found, the software will default to the following keys in this order: 

    ParameterKey(mechanism = "mechanism_type", part_id = "part_id" , name = "parameter_name")
    ParameterKey(mechanism = None, part_id = "part_id" , name = "parameter_name")
    ParameterKey(mechanism = "mechanism_name", part_id = None , name = "parameter_name")
    ParameterKey(mechanism = "mechanism_type", part_id = None , name = "parameter_name")
    ParameterKey(mechanism =  None, part_id = None , name = "parameter_name")
    
As a note, mechanism_name refers to the .name variable of a Mechanism. mechanism_type refers to the .type variable of a Mechanism. Either of these can be used as a mechanism_id. This allows for models to be constructed easily using default parameter values and for parameters to be shared between different Mechanisms and/or Components.

#### Units in BioCRNpyler

The Parameter database in BioCRNpyler can also be used to import units for model parameters. Simply write your units in the parameter file (.csv/.tsv) as a new column titled "unit". To manually create a Parameter object with a unit, you can run:
```
    new_parameter = Parameter(parameter_name="None", parameter_value="1.0", unit = "M")
    > new_paramter.unit == "M"
    > True
```
BioCRNpyler supports the following unit definitions. The string in paranthesis is the unit identifier that you should use for that particular unit: 
1. Time units: second ("second"), minute ("minute"), hour ("hour").
2. Concentration units: nanomolar ("nM"), micromolar ("uM"), millimolar ("mM"), molar ("M"). 
3. Volume units: nanolitre ("nL"), microlitre ("uL"), millilitre ("mL"), litre ("L").
4. Common parameter units: per second ("per_second"), per hour ("per_hour"), 1/(molar x second) ("litre_per_mole_per_second"), 1/(molar x hour) ("litre_per_mole_per_hour")

To add support for any new unit of your choice, simply go to `units.py` in the `biocrnpyler/` source code and add your own function definition for a unit (you can use the existing functions to get an idea about how to create your own new definition).

#### The ParameterDatabase and ParameterEntries
Each Component and Mixture has its own ParameterDatabase full of ParameterEntries. In general, users are not expected to make these objects by hand. Instead, ParameterDatabases will be automatically populated with ParameterEntries using either a parameter_file or parameter_dictionary. However, they can be accessed and view by looking at Component.parameter_database and Mixture.parameter_database

#### Multiple Parameter Files:
Components and Mixtures can both have one more multiple parameter files by passing in a list of filenames instead of a single filename to the parameter_file keyword. Components use parameters loaded from their file(s) before defaulting to the file(s) supplied to a Mixture. The last file in any list will take precedent and overwrite parameter files which were written earlier.

Below is an example csv with all the parameters for a tetR promoter undergoing Michaelis Menten transcription and translation.

## 1. The Parameter File
In the following cell we look at an example parameter file that will run with no parameter defaulting.

In [1]:
from biocrnpyler import *
perfect_param_file_name = "Perfect Param File Example.tsv"

#Open and print the parameter file
param_file = open(perfect_param_file_name)
print("****Parameter File****")
print(param_file.read())
param_file.close()

****Parameter File****
mechanism_id	part_id	param_name	param_val	unit	comments
transcription_mm	ptet_tetR	kb	10.	litre_per_mole_per_hour	extra columns are okay!
transcription_mm	ptet_tetR	ku	.1	per_hour	These are the parameters for transcription
transcription_mm	ptet_tetR	ktx	1.		
transcription_mm	ptet_leak	ktx	.1		These are the parameters for transcription leak
transcription_mm	ptet_leak	ku	.1		
transcription_mm	ptet_leak	kb	.1		
one_step_cooperative_binding	ptet_tetR	ku	.1		These are parameters for tetR dimerization and binding with the promoter
one_step_cooperative_binding	ptet_tetR	kb	.1		
one_step_cooperative_binding	ptet_tetR	cooperativity	2		
translation_mm	BCD	ktl	2.0		These are parameters for translation
translation_mm	BCD	ku	.25	per_hour	
translation_mm	BCD	kb	10	litre_per_mole_per_hour	
rna_degredation_mm		kb	10		"These are parameters for RNA degredation. "
rna_degredation_mm		ku	.5		They will be the same for all RNAs because of parameter defaulting.
rna_degredation_mm		kdeg

### Use this parameter file to create a CRN

In [2]:
#Create a Regulated Promoter
Ptet = RegulatedPromoter("ptet", regulators=["tetR"], leak=True)
reg_rep_assembly = DNAassembly(name="reporter", promoter=Ptet, rbs="BCD")
tet = Protein("tetR")
components = [reg_rep_assembly, tet]
myMixture = TxTlExtract(name="txtl", parameter_file = perfect_param_file_name, components=components)


#Print the parameter database created from the file
print("\n****Loaded Parameters****")
for param in myMixture.parameter_database:
    print(param.parameter_key, "==>", param.value)

CRN = myMixture.compile_crn()    
print("\n****Resulting CRN****\nNotice that the found_key perfectly matches the search_key.\n")
print(CRN.pretty_print(show_rates = True, show_keys = True))

# Write to SBML
CRN.write_sbml_file('model_with_perfect_params.xml')


****Loaded Parameters****
ParameterKey(mechanism='transcription_mm', part_id='ptet_tetR', name='kb') ==> 10.0
ParameterKey(mechanism='transcription_mm', part_id='ptet_tetR', name='ku') ==> 0.1
ParameterKey(mechanism='transcription_mm', part_id='ptet_tetR', name='ktx') ==> 1.0
ParameterKey(mechanism='transcription_mm', part_id='ptet_leak', name='ktx') ==> 0.1
ParameterKey(mechanism='transcription_mm', part_id='ptet_leak', name='ku') ==> 0.1
ParameterKey(mechanism='transcription_mm', part_id='ptet_leak', name='kb') ==> 0.1
ParameterKey(mechanism='one_step_cooperative_binding', part_id='ptet_tetR', name='ku') ==> 0.1
ParameterKey(mechanism='one_step_cooperative_binding', part_id='ptet_tetR', name='kb') ==> 0.1
ParameterKey(mechanism='one_step_cooperative_binding', part_id='ptet_tetR', name='cooperativity') ==> 2.0
ParameterKey(mechanism='translation_mm', part_id='BCD', name='ktl') ==> 2.0
ParameterKey(mechanism='translation_mm', part_id='BCD', name='ku') ==> 0.25
ParameterKey(mechanism='

True

We will now look at an example of a parameter file that uses defaulting. If you were to fill in this file with full parameter signatures (mechanism_id, part_id, param_name, value), the errors at the bottom of the readout would slowly diminish. However, even without full values the file loads and runs. Although this example uses only the key "param_name" for default values, there exists a heirarchy of keys to allow for shared parameters between different Components and Mechanisms.

The parameter key heirarchy (top takes priority):
1) (mechanism_name, part_id, param_name)
2) (mechanism_type, part_id, param_name)
3) (part_id, param_name)
4) (mechanism_name, param_name)
5) (mechanism_type, param_name)
6) (param_name)

here the column "mechanism_type" can either be a Mechanism's type string Mechanism.type (eg "transcription") or its name string Mechanism.name (eg "transcription_mm"). 

In [8]:
from biocrnpyler import *
default_param_file_name = "Default Param File Example.tsv"

#Open and print the parameter file
param_file = open(default_param_file_name)
print("****Parameter File****")
print(param_file.read())
param_file.close()

****Parameter File****
param_name	param_val	comments	mechanism_id	part_id
ku	1.0	Any parameter called ku will default to this		
kb	1.0	Default kb		
ktx	2.0	Default ktx		
ktl	3.0	Default ktl		
cooperativity	2.0	Default cooperativity		
kdeg	.5	Default degredation		


### And use these parameters in a CRN - not all reactions of the same name share the same parameters!

In [10]:
#Create a Regulated Promoter
Ptet = RegulatedPromoter("ptet", regulators=["tetR"], leak=True)
reg_rep_assembly = DNAassembly(name="reporter", promoter=Ptet, rbs="BCD")
tet = Protein("tetR")
components = [reg_rep_assembly, tet]

myMixture = TxTlExtract(name="txtl", parameter_file = default_param_file_name, components=components)

#Print the parameter dictionary created from the file
print("\n****Loaded Parameters****")
for param in myMixture.parameter_database:
    print(param.parameter_key, "==>", param.value)
    
print("\n****Resulting CRN****\nNotice that only the name portion of parameter keys match.\n")
print(myMixture.compile_crn().pretty_print(show_rates = True, show_keys = True))


****Loaded Parameters****
ParameterKey(mechanism=None, part_id=None, name='ku') ==> 1.0
ParameterKey(mechanism=None, part_id=None, name='kb') ==> 1.0
ParameterKey(mechanism=None, part_id=None, name='ktx') ==> 2.0
ParameterKey(mechanism=None, part_id=None, name='ktl') ==> 3.0
ParameterKey(mechanism=None, part_id=None, name='cooperativity') ==> 2.0
ParameterKey(mechanism=None, part_id=None, name='kdeg') ==> 0.5

****Resulting CRN****
Notice that only the name portion of parameter keys match.

Species (12) = {0. [dna[reporter]:2x_protein[tetR]], 1. complex[[dna[reporter]:2x_protein[tetR]]:protein[RNAP]], 2. protein[tetR], 3. protein[RNAP], 4. protein[reporter], 5. complex[protein[RNAase]:rna[reporter]], 6. complex[dna[reporter]:protein[RNAP]], 7. protein[RNAase], 8. rna[reporter], 9. complex[protein[Ribo]:rna[reporter]], 10. dna[reporter], 11. protein[Ribo]}
Reactions (9) = [
0. dna[reporter]+protein[RNAP] <--> complex[dna[reporter]:protein[RNAP]]
 Kf=k_forward * dna_reporter * protein_R

### Setting Parameters at the component level
Components can have their own parameter files instead of relying on the parameter files passed into a mixture. Components can also have the mixtures default parameters overwritten with a parameter dictionary. Below, we will create a DNAassembly which has custom parameters loaded in as a dictionary (this works the same as loading them with a file). We will put this in a Mixture with the default parameters from the above example. This example also helps illustrate how the parameter loading heirarchy works if you examine the final CRNs.

In [5]:
#Create custom parameter dictionary for the ptet promoter. In this case, we will add specific leak parameters and
ra_param_dict = {
    ParameterKey(mechanism = "transcription_mm", part_id = None, name = "ku"):33.33, #These parameters will take priority over single key parameters
    ParameterKey(mechanism = "transcription_mm", part_id = None, name = "kb"):.3333, 
    ParameterKey(mechanism = "transcription_mm", part_id = None, name = "ktx"):3.333, 
    ParameterKey(mechanism = "transcription_mm", part_id = "ptet_leak", name = "ku"):111.1, #these parameters will take priority over the ones above
    ParameterKey(mechanism = "transcription_mm", part_id = "ptet_leak", name = "kb"):.1111, 
}

#Use the parameter_file keyword to update the parameters with a file.
#Use the parameters keyword to update the parameters with a dictionary
#If Use both: the dictionary takes precedent to the file if there are conflicts.
Ptet = RegulatedPromoter("ptet", regulators=["tetR"], leak=True, parameters = ra_param_dict)
reg_rep_assembly = DNAassembly(name="reporter", promoter=Ptet, rbs="BCD")
tet = Protein("tetR")
components = [reg_rep_assembly, tet]
myMixture = TxTlExtract(name="txtl", parameter_file = default_param_file_name, components=components)

### Mixture parameters come from the file

In [6]:
print("\n****Mixture Parameters****")
for param in myMixture.parameter_database:
    print(param.parameter_key, "==>", param.value)


****Mixture Parameters****
ParameterKey(mechanism=None, part_id=None, name='ku') ==> 1.0
ParameterKey(mechanism=None, part_id=None, name='kb') ==> 1.0
ParameterKey(mechanism=None, part_id=None, name='ktx') ==> 2.0
ParameterKey(mechanism=None, part_id=None, name='ktl') ==> 3.0
ParameterKey(mechanism=None, part_id=None, name='cooperativity') ==> 2.0
ParameterKey(mechanism=None, part_id=None, name='kdeg') ==> 0.5


### Component level parameters come from the dictionary

In [9]:
### Component Parameters are different!
print("\n****Ptet Parameters****")
for param in Ptet.parameter_database:
    print(param.parameter_key, "==>", param.value)


****Ptet Parameters****
ParameterKey(mechanism='transcription_mm', part_id=None, name='ku') ==> 33.33
ParameterKey(mechanism='transcription_mm', part_id=None, name='kb') ==> 0.3333
ParameterKey(mechanism='transcription_mm', part_id=None, name='ktx') ==> 3.333
ParameterKey(mechanism='transcription_mm', part_id='ptet_leak', name='ku') ==> 111.1
ParameterKey(mechanism='transcription_mm', part_id='ptet_leak', name='kb') ==> 0.1111


### The Compiled CRN uses Component level parameters if it can find them before defaulting to Mixture level Parameters

In [11]:
myCRN = myMixture.compile_crn()
print(myCRN.pretty_print(show_rates = True, show_keys = True))

Species (12) = {0. [dna[reporter]:2x_protein[tetR]], 1. complex[[dna[reporter]:2x_protein[tetR]]:protein[RNAP]], 2. protein[tetR], 3. protein[RNAP], 4. protein[reporter], 5. complex[protein[RNAase]:rna[reporter]], 6. complex[dna[reporter]:protein[RNAP]], 7. protein[RNAase], 8. rna[reporter], 9. complex[protein[Ribo]:rna[reporter]], 10. dna[reporter], 11. protein[Ribo]}
Reactions (9) = [
0. dna[reporter]+protein[RNAP] <--> complex[dna[reporter]:protein[RNAP]]
 Kf=k_forward * dna_reporter * protein_RNAP
 Kr=k_reverse * complex_dna_reporter_protein_RNAP
  k_forward=1.0
  found_key=(mech=None, partid=None, name=kb).
  search_key=(mech=transcription_mm, partid=ptet_leak, name=kb.
  k_reverse=1.0
  found_key=(mech=None, partid=None, name=ku).
  search_key=(mech=transcription_mm, partid=ptet_leak, name=ku.

1. complex[dna[reporter]:protein[RNAP]] --> dna[reporter]+rna[reporter]+protein[RNAP]
 Kf=k_forward * complex_dna_reporter_protein_RNAP
  k_forward=2.0
  found_key=(mech=None, partid=None,